In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

sns.set(style='whitegrid')


In [ ]:
# Create SQLite databases
jotstar_conn = sqlite3.connect("jotstar.db")
liocinema_conn = sqlite3.connect("liocinema.db")

# Load SQL files
with open("/mnt/data/Jotstar_db.sql", "r") as f:
    jotstar_sql = f.read()
with open("/mnt/data/LioCinema_db.sql", "r") as f:
    liocinema_sql = f.read()

# Execute SQL scripts to create tables and load data
jotstar_conn.executescript(jotstar_sql)
liocinema_conn.executescript(liocinema_sql)


In [ ]:
# List tables in both databases
def list_tables(conn):
    return pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

print("📦 Jotstar Tables")
display(list_tables(jotstar_conn))
print("📦 LioCinema Tables")
display(list_tables(liocinema_conn))


## 🚀 Analysis Starts Here (to be continued with user queries)

### 1. Total Users & Growth Trends

In [ ]:
# Monthly user count (Jan to Nov 2024)
query = '''
SELECT strftime('%m', created_at) AS month,
       COUNT(*) AS user_count
FROM subscribers
WHERE created_at BETWEEN '2024-01-01' AND '2024-11-30'
GROUP BY month
ORDER BY month;
'''
lio_growth = pd.read_sql(query, liocinema_conn)
jot_growth = pd.read_sql(query, jotstar_conn)

lio_growth['Platform'] = 'LioCinema'
jot_growth['Platform'] = 'Jotstar'
all_growth = pd.concat([lio_growth, jot_growth])

px.line(all_growth, x='month', y='user_count', color='Platform',
        title='📈 Monthly User Growth (Jan–Nov 2024)')


### 2. Content Library Comparison

In [ ]:
# Total content count and breakdown
query = '''
SELECT content_type, language, COUNT(*) AS count
FROM content
GROUP BY content_type, language
ORDER BY count DESC;
'''
lio_content = pd.read_sql(query, liocinema_conn)
jot_content = pd.read_sql(query, jotstar_conn)

lio_content['Platform'] = 'LioCinema'
jot_content['Platform'] = 'Jotstar'
content_df = pd.concat([lio_content, jot_content])

content_df.head(10)


### 3. User Demographics

In [ ]:
query = '''
SELECT age_group, city_tier, current_plan, COUNT(*) AS user_count
FROM subscribers
GROUP BY age_group, city_tier, current_plan;
'''
lio_demo = pd.read_sql(query, liocinema_conn)
jot_demo = pd.read_sql(query, jotstar_conn)

lio_demo['Platform'] = 'LioCinema'
jot_demo['Platform'] = 'Jotstar'
pd.concat([lio_demo, jot_demo])


### 4. Active vs Inactive Users

In [ ]:
query = '''
SELECT is_active, age_group, current_plan, COUNT(*) AS count
FROM subscribers
GROUP BY is_active, age_group, current_plan;
'''
lio_status = pd.read_sql(query, liocinema_conn)
jot_status = pd.read_sql(query, jotstar_conn)

lio_status['Platform'] = 'LioCinema'
jot_status['Platform'] = 'Jotstar'
pd.concat([lio_status, jot_status])


### 5. Watch Time Analysis

In [ ]:
query = '''
SELECT device_type, ROUND(AVG(total_watch_time_mins), 2) AS avg_watch_time
FROM content_consumption
GROUP BY device_type;
'''
lio_watch = pd.read_sql(query, liocinema_conn)
jot_watch = pd.read_sql(query, jotstar_conn)

lio_watch['Platform'] = 'LioCinema'
jot_watch['Platform'] = 'Jotstar'
pd.concat([lio_watch, jot_watch])


### 6. Inactivity Correlation

In [ ]:
query = '''
SELECT s.is_active, AVG(c.total_watch_time_mins) AS avg_watch
FROM content_consumption c
JOIN subscribers s ON c.user_id = s.user_id
GROUP BY s.is_active;
'''
lio_corr = pd.read_sql(query, liocinema_conn)
jot_corr = pd.read_sql(query, jotstar_conn)

lio_corr['Platform'] = 'LioCinema'
jot_corr['Platform'] = 'Jotstar'
pd.concat([lio_corr, jot_corr])


### 7. Downgrade Trends

In [ ]:
query = '''
SELECT from_plan, to_plan, COUNT(*) AS downgrade_count
FROM plan_transitions
WHERE downgrade = 1
GROUP BY from_plan, to_plan
ORDER BY downgrade_count DESC;
'''
lio_down = pd.read_sql(query, liocinema_conn)
jot_down = pd.read_sql(query, jotstar_conn)

lio_down['Platform'] = 'LioCinema'
jot_down['Platform'] = 'Jotstar'
pd.concat([lio_down, jot_down])


### 8. Upgrade Patterns

In [ ]:
query = '''
SELECT from_plan, to_plan, COUNT(*) AS upgrade_count
FROM plan_transitions
WHERE upgrade = 1
GROUP BY from_plan, to_plan
ORDER BY upgrade_count DESC;
'''
lio_up = pd.read_sql(query, liocinema_conn)
jot_up = pd.read_sql(query, jotstar_conn)

lio_up['Platform'] = 'LioCinema'
jot_up['Platform'] = 'Jotstar'
pd.concat([lio_up, jot_up])


### 9. Paid User Distribution by City Tier

In [ ]:
query = '''
SELECT city_tier, current_plan, COUNT(*) AS users
FROM subscribers
WHERE current_plan NOT IN ('Free')
GROUP BY city_tier, current_plan;
'''
lio_paid = pd.read_sql(query, liocinema_conn)
jot_paid = pd.read_sql(query, jotstar_conn)

lio_paid['Platform'] = 'LioCinema'
jot_paid['Platform'] = 'Jotstar'
pd.concat([lio_paid, jot_paid])


### 10. Revenue Analysis

In [ ]:
# Revenue assumptions
plan_prices = {
    'Free': 0,
    'Basic': 199,
    'VIP': 299,
    'Premium': 499
}

query = '''
SELECT current_plan, COUNT(*) AS subscribers
FROM subscribers
GROUP BY current_plan;
'''
lio_rev = pd.read_sql(query, liocinema_conn)
jot_rev = pd.read_sql(query, jotstar_conn)

def calc_revenue(df, months):
    df['price'] = df['current_plan'].map(plan_prices)
    df['revenue'] = df['subscribers'] * df['price'] * months
    return df[['current_plan', 'revenue']]

lio_rev = calc_revenue(lio_rev, 11)
jot_rev = calc_revenue(jot_rev, 11)

lio_rev['Platform'] = 'LioCinema'
jot_rev['Platform'] = 'Jotstar'
pd.concat([lio_rev, jot_rev])
